# 국민재난안전포털 웹 크롤러 만들기

In [7]:
from requests import Session
from requests.compat import urljoin
from bs4 import BeautifulSoup

In [59]:
session = Session()

base_url = 'http://www.safekorea.go.kr/'
main_url = 'idsiSFK/neo/ext/json/main/menuList.json'

main_resp = session.get(urljoin(base_url, main_url))
result = main_resp.json()

disaster_service_url = ''
disaster_menu_seq = 1

# 재난문자 url과 params인 munuSeq 구하기
for cnt in range(0,len(result)):
    if cnt > 0 and result[cnt]['MENU_NM'] == '재난문자':
        disaster_service_url = 'idsiSFK' + result[cnt]['SERVICE_URL']
        disaster_menu_seq = result[cnt]['MENU_SEQ']
        break
        
#disaster_service_url, disaster_menu_seq

## 아래 코드 사용 불가
- post 방식의 재난문자 json request시 500 error 발생

In [75]:
session.cookies.clear()
disaster_resp = session.request('GET', urljoin(base_url, disaster_service_url), params={'menuSeq':disaster_menu_seq})
#dom = BeautifulSoup(disaster_resp.text, 'html.parser')
referer = disaster_resp.request.url

#새 쿠키값 
cookiedict = {}
for _ in session.cookies.items():
    cookiedict[_[0]] = _[1]

headers = {
    'Referer' : referer,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

#쿠키 날짜 value는 당일 날짜로 변경해야함, 실시간 크롤링시
cookies = {
    'JSESSIONID':cookiedict['JSESSIONID'],
    'elevisor_for_j2ee_uid':cookiedict['elevisor_for_j2ee_uid'],
    'e4d3f82b588e6cf132ee4765403cd800':cookiedict['e4d3f82b588e6cf132ee4765403cd800'],
    '3053195ebb01870828aed202f55dc974':'76ff93199833d41c33e1b89a9862350b',
    'popup_1':'20200802',
    'popup_2':'20200802',
    '_ga':'GA1.3.522710010.1595930482',
    '_gid':'GA1.3.1758721736.1596362546'
}

#session.request지만 누락된 쿠키값이 있어서 따로 다시 입력
disaster_list = 'idsiSFK/bbs/user/selectBbsList.do'
disaster_list_resp = session.request('POST', urljoin(base_url, disaster_list), headers=headers, cookies=cookies)

In [76]:
disaster_list_resp.request, disaster_list_resp.reason, disaster_list_resp.status_code, disaster_list_resp.headers

(<PreparedRequest [POST]>,
 'Internal Server Error',
 500,
 {'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '254', 'Date': 'Sun, 02 Aug 2020 14:56:23 GMT'})

# 셀레니움 활용 실시간 문자 수집
- 셀레니움 사용 이유: 재난문자 페이지의 json request 불가(inter server error, 500 error)

In [77]:
from selenium import webdriver

In [83]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

uu = urljoin(base_url, disaster_service_url)
uuu = uu + '&menuSeq=' + str(disaster_menu_seq)
driver.get(uuu)

#### (수정1) 재난문자 첫 페이지 => 새로운 재난문자 생길 경우 => 새로운 문자만 접근
- list에 첫 페이지의 모든 재난문자 제목 insert
- 맨 밑의 문자부터 가장 최근 문자 순으로 체크
- 새로운 문자가 있을 경우, 가장 나중에 갱신된 문자~처음 갱신된 문자까지 접근

#### (처음) 재난문자 첫 페이지 => 10개 재난문자 가져오기
- 재난문자 하나씩 접근하므로 중간에 재난문자 갱신이 되면 똑같은 문자 접근할 수 있음

In [156]:
import time

run_count = 0
last_chk_topic = []
new_chk_topic = []
def run_chk_msg():
    global run_count
    global chk_topic
    
    print('run_count: {0}'.format(run_count))
    print('chk_msg!')
    
    for idx in range(0, 10): # last_재난문자
        chk_msg = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(idx)
        last_chk_topic.insert(idx, driver.find_element_by_xpath(chk_msg).text)
        print(last_chk_topic[idx])
        
    # 현재 켜져 있는 웹페이지에서 새로고침 후 새로운 재난문자 접근
    # 나중에 기준 정하기. 웹 서비스로 처음 시각화되는 재난문자를 어떤 것으로 둘지(지금은 시각화되고 있음을 가정)
    driver.refresh()
    new_msg_idx = -1 # 바뀐 문자 index
    for idx in range(9, -1, -1): # new_재난문자 체크
        cur_msg_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(idx)
        cur_msg = driver.find_element_by_xpath(cur_msg_path)
        new_exist = True
        for chk in range(9, -1, -1):
            if last_chk_topic[chk] == cur_msg.text: # 같은 재난문자이면 break
                new_exist = False
                break
        if new_exist == True:
            # 재난문자가 새로 발신된 경우, for문 break
            new_msg_idx = idx
            print('new 재난문자: {0}개'.format(new_msg_idx+1))
            break
    
    print('new_msg_idx: {0}'.format(new_msg_idx))
    if new_msg_idx >= 0: # 새로운 문자가 발신된 경우
        for cnt in range(0, new_msg_idx+1):
            # 첫 번째 페이지, cnt 번째 재난 문자 path
            disaster_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(cnt)
            result_topic = driver.find_element_by_xpath(disaster_path)
            print((cnt+1), '번째 재난문자:', result_topic.text)

            # cnt 번째 재난문자 링크 이동
            first_a = disaster_path + '/a'
            last_url = driver.current_url
            driver.find_element_by_xpath(first_a).click()
            driver.implicitly_wait(3)

            # 재난문자 상세내용 확인
            disaster_context = '//div[@id="content"]/div[@id="content"]/div/div[@class="boardView"]/div[@class="boardView_context"]'
            result_context = driver.find_element_by_xpath(disaster_context)
            print(result_context.text)
            print('\n')
            driver.implicitly_wait(3)

            # 이전 페이지로 돌아가기
            driver.get(last_url)
            
    run_count += 1

In [157]:
while True:
    if run_count > 0:
        time.sleep(30)
    run_chk_msg()
#     if run_count > 50:
#         print('end')
#         break

run_count: 0
chk_msg!
2020/08/04 15:44:18 재난문자[화천군청]
2020/08/04 15:27:48 재난문자[세종시청]
2020/08/04 15:14:52 재난문자[철원군청]
2020/08/04 15:13:24 재난문자[양구군청]
2020/08/04 15:12:54 재난문자[양구군청]
2020/08/04 15:07:15 재난문자[인제군청]
2020/08/04 14:50:12 재난문자[고창군청]
2020/08/04 14:45:05 재난문자[천안시청]
2020/08/04 14:22:15 재난문자[제천시청]
2020/08/04 14:21:48 재난문자[화천군청]
new 재난문자: 2개
new_msg_idx: 1
1 번째 재난문자: 2020/08/04 16:11:14 재난문자[철원군청]
[철원군청]금일 근남면 육단리 육단2교의 범람이 우려되오니 도로 통행제한 중에 있습니다. 도로 침수되여 통행이 어려우니 우회바랍니다.

-송출지역-
강원도 철원군


2 번째 재난문자: 2020/08/04 15:56:00 재난문자[화천군청]
[화천군청] 금일 상서면 다목리 수피령고개 일원 집중호우로 통행제한 중에 있습니다. 도로 침수 및 토사유출로 통행이 어려우니 우회바랍니다.

-송출지역-
강원도 화천군


run_count: 1
chk_msg!
2020/08/04 16:11:14 재난문자[철원군청]
2020/08/04 15:56:00 재난문자[화천군청]
2020/08/04 15:44:18 재난문자[화천군청]
2020/08/04 15:27:48 재난문자[세종시청]
2020/08/04 15:14:52 재난문자[철원군청]
2020/08/04 15:13:24 재난문자[양구군청]
2020/08/04 15:12:54 재난문자[양구군청]
2020/08/04 15:07:15 재난문자[인제군청]
2020/08/04 14:50:12 재난문자[고창군청]
2020/08/04 14:45:05 재난문자[천안시청]
new_msg_idx: -1
run_count: 2
chk_ms

KeyboardInterrupt: 

# 재난문자 전처리 과정(진행중)
- 지역 분류
- 재난 카테고리 분류
- 재난 심각도 상/중/하 기준
##### 문제점: 문자 내용 확인하러 링크 들어간 후, 새로운 문자 생성시 같은 문자 접근 가능성 있음
##### 해결 방안: 시각화되고 있는 재난문자들을 기억해두고, 해당 문자의 key값으로 찾아간다.(pk: 번호 + 제목) => 다음 페이지로 넘어가는 것도 고려해야 함(지금은 첫 페이지만 대상)

In [158]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [166]:
topic_data = []
context_data = []

for cnt in range(0, 10):
    disaster_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(cnt)
    result_topic = driver.find_element_by_xpath(disaster_path)
    topic_data.insert(cnt, result_topic.text)
    print((cnt+1), '번째 재난문자:', topic_data[cnt])

    # cnt 번째 재난문자 링크 이동
    first_a = disaster_path + '/a'
    last_url = driver.current_url
    driver.find_element_by_xpath(first_a).click()
    driver.implicitly_wait(3)

    # 재난문자 상세내용 확인
    disaster_context = '//div[@id="content"]/div[@id="content"]/div/div[@class="boardView"]/div[@class="boardView_context"]'
    result_context = driver.find_element_by_xpath(disaster_context)
    context_data.insert(cnt, result_context.text)
    print(context_data[cnt])
    print('\n')
    driver.implicitly_wait(3)

    # 이전 페이지로 돌아가기
    driver.get(last_url)

1 번째 재난문자: 2020/08/05 11:35:09 재난문자[증평군청]
[증평군청]청주시 신율봉공원 7.31. 종교모임 참석자는 코로나19 검사를 꼭 받으시고 선별진료소 방문시 대중교통이용은 불가하며 마스크 착용 후 방문 바랍니다.

-송출지역-
충청북도 증평군


2 번째 재난문자: 2020/08/05 11:37:17 재난문자[보성군]
[보성군]최근 카페,음식점 등 일상적 공간에서 감염사례 발생, 실내에서도 2m거리두기와 대화자제, 대화시 마스크 착용은 필수입니다. 서로에게 안전망이 되어주세요.

-송출지역-
전라남도 보성군


3 번째 재난문자: 2020/08/05 11:36:42 재난문자[순천시청]
[순천시청]확진자가 자가용이용 및 마스크착용 후 방문(7.25 / 7.27)한 시설(주암면 000마켓)은 방역소독 완료, 임시폐쇄조치 하였으니 안심하시기 바랍니다

-송출지역-
전라남도 순천시


4 번째 재난문자: 2020/08/05 11:35:09 재난문자[증평군청]
[증평군청]청주시 신율봉공원 7.31. 종교모임 참석자는 코로나19 검사를 꼭 받으시고 선별진료소 방문시 대중교통이용은 불가하며 마스크 착용 후 방문 바랍니다.

-송출지역-
충청북도 증평군


5 번째 재난문자: 2020/08/05 11:34:18 재난문자[가평군청]
[가평군청] 계속된 집중호우로 인한 지반약화로 산사태, 저지대 침수, 도로유실 등이 발생되고 있으니 안전한 곳으로 대피하시기 바랍니다.

-송출지역-
경기도 가평군


6 번째 재난문자: 2020/08/05 11:33:03 재난문자[과천시청]
[과천시청] 과천 13번 확진자 발생, 시설격리 중 확진되어 접촉자 및 관내 동선은 없음. 환자 이송 및 방역소독 완료.

-송출지역-
경기도 과천시


7 번째 재난문자: 2020/08/05 11:21:45 재난문자[고성군청]
[고성군청] 현재 고성군 많은 비로 하천 수위 상승, 토사유출 등 위험 우려가 있으니 산사태, 상습침구 등 위험지역대피, 외출자제 등 안전에 주의바랍니다.

-

#### 실시간으로 수집한 new 재난문자를 기존의 분류한 사전(dict)을 활용하여 new 재난문자 분류하기

In [177]:
for idx in range(0, 10):
    print(word_tokenize(topic_data[idx]))
    print(word_tokenize(context_data[idx]))
    print('\n')

['2020/08/05', '11:35:09', '재난문자', '[', '증평군청', ']']
['[', '증평군청', ']', '청주시', '신율봉공원', '7.31', '.', '종교모임', '참석자는', '코로나19', '검사를', '꼭', '받으시고', '선별진료소', '방문시', '대중교통이용은', '불가하며', '마스크', '착용', '후', '방문', '바랍니다', '.', '-송출지역-', '충청북도', '증평군']


['2020/08/05', '11:37:17', '재난문자', '[', '보성군', ']']
['[', '보성군', ']', '최근', '카페', ',', '음식점', '등', '일상적', '공간에서', '감염사례', '발생', ',', '실내에서도', '2m거리두기와', '대화자제', ',', '대화시', '마스크', '착용은', '필수입니다', '.', '서로에게', '안전망이', '되어주세요', '.', '-송출지역-', '전라남도', '보성군']


['2020/08/05', '11:36:42', '재난문자', '[', '순천시청', ']']
['[', '순천시청', ']', '확진자가', '자가용이용', '및', '마스크착용', '후', '방문', '(', '7.25', '/', '7.27', ')', '한', '시설', '(', '주암면', '000마켓', ')', '은', '방역소독', '완료', ',', '임시폐쇄조치', '하였으니', '안심하시기', '바랍니다', '-송출지역-', '전라남도', '순천시']


['2020/08/05', '11:35:09', '재난문자', '[', '증평군청', ']']
['[', '증평군청', ']', '청주시', '신율봉공원', '7.31', '.', '종교모임', '참석자는', '코로나19', '검사를', '꼭', '받으시고', '선별진료소', '방문시', '대중교통이용은', '불가하며', '마스크', '착용', '후', '방문', '바랍니다', '.', '-송출지역-', '

#### api 재난문자 데이터, 문자의 단어 빈도수 측정
- 세부적인 전처리 필요
- TF-IDF만으로 쳐내는 데이터 중엔 필요 없는 데이터만 존재할지(분석필요)
- api 데이터와 join해서 사용할 데이터 생각(재난의 위험성, 재난 분류 등에 필요할 것 같음)

In [189]:
import pandas as pd
from collections import Counter

In [195]:
dd = pd.read_csv('msg_data.csv')
word_list = []
for cnt in range(0, len(dd)):
    word_token = word_tokenize(dd.Msg[cnt]) # 단어의 전처리 더욱 세부적으로 필요
    for _ in range(0, len(word_token)):
        word_list.append(word_token[_])

dd

,Date,LocationId,LocationName,Msg,Number
0,2020/08/05 10:28:29,14,강원도 철원군,"[철원군청] 오늘 09시53분 집중호우로 한탄천 범람, 갈말읍 정연교-삼합교 일대 ...",42355
1,2020/08/05 10:24:14,235,충청남도 청양군,[청양군청] 오늘 강한 바람과 비가 예상됩니다.▲비닐하우스 고정▲배수구 정비▲위험수...,42354
2,2020/08/05 10:22:06,54,경상남도 거제시,"[거제시청] 22:30마산의료원이송(앰뷸런스)/자택 방역완료, 접촉자3명(남편, 자...",42353
3,2020/08/05 10:21:54,54,경상남도 거제시,[거제시청] 8.3(월) 9:00거제시보건소선별진료소(자차) 10:00자택 8.4(...,42352
4,2020/08/05 10:21:32,54,경상남도 거제시,[거제시청]19번 확진자 발생(프랑스국적 외국인/30대/여/프랑스체류 후 입국) 8...,42351
...,...,...,...,...,...
16995,2020/02/12 15:00:29,222,충청남도 전체,"[충남도청]신종코로나바이러스 예방을 위해 손씻기, 기침예절준수와 14일내 중국방문 ...",25360
16996,2020/02/12 14:17:26,196,전라남도 장성군,"[장성군]해외방문 후 발열, 호흡기 이상 발생시 장성군보건소(390-8333), 콜...",25359
16997,2020/02/12 13:54:04,216,전라북도 진안군,"[진안군청] 해외여행 14일 이내 발열(37.5도 이상), 호흡기 증상 시 진안군의...",25358
16998,2020/02/12 13:30:07,208,전라북도 부안군,[부안군청]부안군은 코로나바이러스 유입이없으니 안심하시기 바랍니다. 개인위생준수. ...,25357


In [196]:
frequency = Counter(word_list).most_common()
print(frequency)

[(',', 23782), ('.', 19239), (')', 18351), ('(', 18298), ('[', 18047), (']', 18039), ('확진자', 5055), ('바랍니다', 4595), ('코로나19', 4137), ('및', 3705), ('등', 3497), ('발생', 2621), ('마스크', 1829), ('홈페이지', 1623), ('주시기', 1511), ('위해', 1363), ('보건소', 1249), ('후', 1217), ('동선', 1175), (':', 1175), ('착용', 1155), ('거리두기', 1101), ('방문', 971), ('시', 964), ('자제', 914), ('로', 908), ('>', 899), ('손씻기', 869), ('중', 854), ('역학조사', 836), ('1명', 831), ('사회적', 817), ('현재', 770), ('동선은', 730), ('추가', 704), ('수', 695), ('마스크착용', 668), ('오늘', 665), ('블로그', 647), ('예방을', 638), ('준수', 634), ('자택', 626), ('자가격리', 606), ('외출', 602), ('발열', 581), ('또는', 557), ('적극', 547), ('거주', 545), ('반드시', 545), ('상담바랍니다', 534), ('없음', 534), ('관련', 530), ('광주광역시', 530), ('접촉자', 525), ('코로나', 524), ('있습니다', 501), ('SNS', 483), ('토', 482), ('금', 479), ('모임', 475), ('자세한', 464), ('월', 462), ('분은', 457), ('방문자는', 457), ('외출자제', 441), ('주시고', 435), ('보건소로', 434), ('타지역', 433), ('방역완료', 432), ('공개', 428), ('연락바랍니다', 426), ('안내', 426), 